In [ ]:
# kenya_eda.ipynb

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

# For wind rose plot
!pip install windrose
from windrose import WindroseAxes

# Load data
df = pd.read_csv("data/kenya.csv", parse_dates=["Timestamp"])

# --- 1. Summary Statistics & Missing Values ---
print("Summary Statistics:\n", df.describe())
print("\nMissing Values:\n", df.isna().sum())

# List columns with >5% missing
missing_percent = (df.isna().sum() / len(df)) * 100
print("\nColumns with >5% missing values:")
print(missing_percent[missing_percent > 5])

# --- 2. Outlier Detection & Basic Cleaning ---

# Columns to check for outliers
cols_outliers = ["GHI", "DNI", "DHI", "ModA", "ModB", "WS", "WSgust"]

# Compute Z-scores
for col in cols_outliers:
    df[col + "_zscore"] = zscore(df[col].fillna(df[col].median()))

# Flag rows with |Z|>3 in any of the columns
outliers = df[(df[[col + "_zscore" for col in cols_outliers]].abs() > 3).any(axis=1)]
print(f"\nNumber of outlier rows: {len(outliers)}")

# Drop outlier rows or alternatively you can keep and impute
df_clean = df.copy()

# Impute missing values in key columns with median
for col in cols_outliers:
    median_val = df_clean[col].median()
    df_clean[col] = df_clean[col].fillna(median_val)

# Drop zscore columns after use
df_clean.drop(columns=[col + "_zscore" for col in cols_outliers], inplace=True)

# --- 3. Export cleaned DataFrame ---
df_clean.to_csv("data/kenya_clean.csv", index=False)

# --- 4. Time Series Analysis ---

plt.figure(figsize=(15,6))
plt.plot(df_clean['Timestamp'], df_clean['GHI'], label='GHI')
plt.plot(df_clean['Timestamp'], df_clean['DNI'], label='DNI')
plt.plot(df_clean['Timestamp'], df_clean['DHI'], label='DHI')
plt.plot(df_clean['Timestamp'], df_clean['Tamb'], label='Tamb')
plt.legend()
plt.title('Time Series of Solar Radiation and Temperature')
plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.show()

# Patterns by month
df_clean['Month'] = df_clean['Timestamp'].dt.month
monthly_avg = df_clean.groupby('Month')[['GHI', 'DNI', 'DHI', 'Tamb']].mean()

monthly_avg.plot(kind='bar', figsize=(12,6))
plt.title('Average Monthly Solar Radiation and Temperature')
plt.xlabel('Month')
plt.ylabel('Average Value')
plt.show()

# --- 5. Cleaning Impact on ModA & ModB ---

cleaned_group = df_clean.groupby('Cleaning')[['ModA', 'ModB']].mean()
print("\nAverage ModA & ModB by Cleaning Status:\n", cleaned_group)

cleaned_group.plot(kind='bar')
plt.title('Effect of Cleaning on Module Sensor Measurements')
plt.xlabel('Cleaning (0=No, 1=Yes)')
plt.ylabel('Average Sensor Reading')
plt.show()

# --- 6. Correlation & Relationship Analysis ---

corr_cols = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']
corr = df_clean[corr_cols].corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# Scatter plots

plt.figure(figsize=(14,6))
plt.subplot(1,3,1)
sns.scatterplot(x='WS', y='GHI', data=df_clean)
plt.title('WS vs GHI')

plt.subplot(1,3,2)
sns.scatterplot(x='WSgust', y='GHI', data=df_clean)
plt.title('WSgust vs GHI')

plt.subplot(1,3,3)
sns.scatterplot(x='WD', y='GHI', data=df_clean)
plt.title('WD vs GHI')

plt.tight_layout()
plt.show()

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.scatterplot(x='RH', y='Tamb', data=df_clean)
plt.title('Relative Humidity vs Ambient Temperature')

plt.subplot(1,2,2)
sns.scatterplot(x='RH', y='GHI', data=df_clean)
plt.title('Relative Humidity vs GHI')

plt.tight_layout()
plt.show()

# --- 7. Wind & Distribution Analysis ---

# Wind rose plot
plt.figure(figsize=(8,8))
ax = WindroseAxes.from_ax()
ax.bar(df_clean['WD'], df_clean['WS'], normed=True, opening=0.8, edgecolor='white')
ax.set_title('Wind Rose (Wind Direction vs Speed)')
ax.set_legend()
plt.show()

# Histograms for GHI and WS
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.histplot(df_clean['GHI'], bins=30, kde=True)
plt.title('Distribution of GHI')

plt.subplot(1,2,2)
sns.histplot(df_clean['WS'], bins=30, kde=True)
plt.title('Distribution of Wind Speed')

plt.show()

# --- 8. Temperature Analysis & Bubble Chart ---

plt.figure(figsize=(10,6))
scatter = plt.scatter(df_clean['GHI'], df_clean['Tamb'], s=df_clean['RH']*2, c=df_clean['BP'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Barometric Pressure (hPa)')
plt.xlabel('GHI (W/m²)')
plt.ylabel('Ambient Temperature (°C)')
plt.title('Bubble Chart: GHI vs Temperature (Bubble size=RH, color=BP)')
plt.show()
